<a href="https://colab.research.google.com/github/chainiqedu/chainiqedu/blob/main/volmex_user_bucketing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Volmex User Bucketing V2

Looking at user's (addresses) that have interacted with the protocol, bucket these into 3 groups e.g Low/Medium/High

Do this by considering users across all 3 chains and look at: Minters, LP's (Liquidity Providers) and Traders (users/addresses that performed swaps in the uniswap/quickswap pools).

## Summary

**Minters**

Unique Minters: 5595  

Unique Minters Split by Chain  
	- Ethereum: 764  
	- Polygon: 4724  
	- Arbitrum: 135  

Cumulative Number of Mints Groupings  
	- Low 0-2: 5438 minters  
	- Medium 2-7: 155 minters  
	- High 7+: 2 minters  

Total Value Collateralized Groupings  
	- Low 0-\$750: 5092 minters  
	- Medium \$750-\$1750: 387 minters  
	- High \$1750+: 116 minters  


**Traders**

Unique Traders: 1391  

Unique Traders Split by Chain  
	- Ethereum: 147  
	- Polygon: 1212  
	- Arbitrum: 39  

Cumulative Trades Groupings  
	- Low 0-9: 1412 traders  
	- Medium 9-40:16 traders  
	- High 40+:11 traders  

Total Value Traded Groupings  
	- Low 0-\$2000: 1393 traders  
	- Medium \$2000-\$10000: 27 traders  
	- High \$10000+: 19 traders  

Cumulative Trading Volume: \$997678.2132998797  
	- **Caveats**: Static price values for the Volatility tokens from coingecko have been used to calculate the USD trade value for tokens, ideally we would have a price history. Therefore depending on whether in the past the Volatility token price is currently lower/higher we may be under-reporting/over-reporting trade value however as an approximate and due to the limitations on working with etherscan these static values are used (e.g https://www.coingecko.com/en/coins/bitcoin-volatility-index-token)  

Cumulative Trading Volume Per Chain  
	- Ethereum: \$780481.8719295093  
	- Polygon: \$211855.5641159281  
	- Arbitrum: \$5340.777254442244   


**Liquidity Providers**

Unique Liquidity Providers: 1685  

Unique Liquidity Providers Split by Chain  
	- Ethereum: 165  
	- Polygon: 1464  
	- Arbitrum: 71  

Cumulative Number of Liquidity Added Actions Groupings  
	- Low 0-2: 1040 LPs  
	- Medium 2-7: 600 LPs  
	- High 7-16: 45 LPs  

Total Liquidity Added Values Groupings  
	- Low 0-\$1500: 1631 LPs  
	- Medium \$1500-\$8000: 38 LPs  
	- High \$8000+: 16 LPs  

In [125]:
#Import libraries
import pandas as pd
import numpy as np
import plotly.express as px

## Minters - Read CSV Data

Read in CSV data from all Ethereum/Polygon/Arbitrum chains for users that have minted volatility tokens

In [126]:
# read ethereum minter CSV files
eth_net_btcv_dai = pd.read_csv('/content/BTCV_DAI_export-address-token-0x187922d4235D10239b2c6CCb2217aDa724F56DDA.csv', index_col=False)
eth_net_ethv_dai = pd.read_csv('/content/ETHV_DAI_export-address-token-0xa57fC404f69fCE71CA26e26f0A4DF7F35C8cd5C3 (1).csv', index_col=False)
eth_net_btcv_usdc = pd.read_csv('/content/BTCV_USDC_export-address-token-0x054FBeBD2Cb17205B57fb56a426ccc54cAaBFaBC.csv', index_col=False)
eth_net_ethv_usdc = pd.read_csv('/content/ETHV_USDC_export-address-token-0x1BB632a08936e17Ee3971E6Eeb824910567e120B.csv', index_col=False)

# read polygon minter CSV files
polygon_btcv_dai = pd.read_csv('/content/BTCV_DAI_export-address-token-0x90E6c403c02f72986a98E8a361Ec7B7C8BC29259.csv', index_col=False)
polygon_ethv_dai = pd.read_csv('/content/ETHV_DAI_export-address-token-0x164c668204Ce54558431997A6DD636Ee4E758b19.csv', index_col=False)
polygon_btcv_usdc = pd.read_csv('/content/BTCV_USDC_export-address-token-0xA2b3501d34edA289F0bEF1cAf95E5D0111032F36.csv', index_col=False)
polygon_ethv_usdc = pd.read_csv('/content/ETHV_USDC_export-address-token-0xEeb6f0C2261E21b657A27582466e5aD9acC072D7.csv', index_col=False)

# read arbitrum minter CSV files
arbitrum_btcv_dai = pd.read_csv('/content/BTCV_DAI_export-address-token-0xe46277336d9cc2ebe7b24ba7268624f5f1495611.csv', index_col=False)
arbitrum_ethv_dai = pd.read_csv('/content/ETHV_DAI_export-address-token-0xf613b55131cf8a69c5b4f62d0d5e5d2c2d9c3280.csv', index_col=False)
arbitrum_btcv_usdc = pd.read_csv('/content/BTCV_USDC_export-address-token-0xdf87072ac4722431861837492edf7adbfec0efa9.csv', index_col=False)
arbitrum_ethv_usdc = pd.read_csv('/content/ETHV_USDC_export-address-token-0xf9b04aad2612d3d664f41e9af5711953e058ff52.csv', index_col=False)

In [127]:
# Add event type marker for collaterlize events to identify minters - when an address/user collaterilzes they mint the volatility tokens (aka minter)
eth_net_btcv_dai['type'] = np.where(eth_net_btcv_dai['To'] == '0x187922d4235d10239b2c6ccb2217ada724f56dda', 'COLLATERALIZE', 'REDEEM/OTHER')
eth_net_ethv_dai['type'] = np.where(eth_net_ethv_dai['To'] == '0xa57fc404f69fce71ca26e26f0a4df7f35c8cd5c3', 'COLLATERALIZE', 'REDEEM/OTHER')
eth_net_btcv_usdc['type'] = np.where(eth_net_btcv_usdc['To'] == '0x054fbebd2cb17205b57fb56a426ccc54caabfabc', 'COLLATERALIZE', 'REDEEM/OTHER')
eth_net_ethv_usdc['type'] = np.where(eth_net_ethv_usdc['To'] == '0x1bb632a08936e17ee3971e6eeb824910567e120b', 'COLLATERALIZE', 'REDEEM/OTHER')

polygon_btcv_dai['type'] = np.where(polygon_btcv_dai['To'] == '0x90e6c403c02f72986a98e8a361ec7b7c8bc29259', 'COLLATERALIZE', 'REDEEM/OTHER')
polygon_ethv_dai['type'] = np.where(polygon_ethv_dai['To'] == '0x164c668204ce54558431997a6dd636ee4e758b19', 'COLLATERALIZE', 'REDEEM/OTHER')
polygon_btcv_usdc['type'] = np.where(polygon_btcv_usdc['To'] == '0xa2b3501d34eda289f0bef1caf95e5d0111032f36', 'COLLATERALIZE', 'REDEEM/OTHER')
polygon_ethv_usdc['type'] = np.where(polygon_ethv_usdc['To'] == '0xeeb6f0c2261e21b657a27582466e5ad9acc072d7', 'COLLATERALIZE', 'REDEEM/OTHER')

arbitrum_btcv_dai['type'] = np.where(arbitrum_btcv_dai['To'] == '0xe46277336d9cc2ebe7b24ba7268624f5f1495611', 'COLLATERALIZE', 'REDEEM/OTHER')
arbitrum_ethv_dai['type'] = np.where(arbitrum_ethv_dai['To'] == '0xf613b55131cf8a69c5b4f62d0d5e5d2c2d9c3280', 'COLLATERALIZE', 'REDEEM/OTHER')
arbitrum_btcv_usdc['type'] = np.where(arbitrum_btcv_usdc['To'] == '0xdf87072ac4722431861837492edf7adbfec0efa9', 'COLLATERALIZE', 'REDEEM/OTHER')
arbitrum_ethv_usdc['type'] = np.where(arbitrum_ethv_usdc['To'] == '0xf9b04aad2612d3d664f41e9af5711953e058ff52', 'COLLATERALIZE', 'REDEEM/OTHER')

In [128]:
# Add Chain Type
eth_net_btcv_dai['chain'] = 'ETHEREUM'
eth_net_ethv_dai['chain'] = 'ETHEREUM'
eth_net_btcv_usdc['chain'] = 'ETHEREUM'
eth_net_ethv_usdc['chain'] = 'ETHEREUM'

polygon_btcv_dai['chain'] = 'POLYGON'
polygon_ethv_dai['chain'] = 'POLYGON'
polygon_btcv_usdc['chain'] = 'POLYGON'
polygon_ethv_usdc['chain'] = 'POLYGON'

arbitrum_btcv_dai['chain'] = 'ARBITRUM'
arbitrum_ethv_dai['chain'] = 'ARBITRUM'
arbitrum_btcv_usdc['chain'] = 'ARBITRUM'
arbitrum_ethv_usdc['chain'] = 'ARBITRUM'

### Minters - Process and Format Data

In [129]:
# Check the TokenSymbols that are part of the ERC20 Transactions
print('ETH BTCV DAI Token Symbol: {}'.format(eth_net_btcv_dai["TokenSymbol"].unique()))
print('ETH ETHV DAI Token Symbol: {}'.format(eth_net_ethv_dai["TokenSymbol"].unique()))
print('ETH BTCV USDC Token Symbol: {}'.format(eth_net_btcv_usdc["TokenSymbol"].unique()))
print('ETH ETHV USDC Token Symbol: {}'.format(eth_net_ethv_usdc["TokenSymbol"].unique()))

print('POLY BTCV DAI Token Symbol: {}'.format(polygon_btcv_dai["TokenSymbol"].unique()))
print('POLY ETHV DAI Token Symbol: {}'.format(polygon_ethv_dai["TokenSymbol"].unique()))
print('POLY BTCV USDC Token Symbol: {}'.format(polygon_btcv_usdc["TokenSymbol"].unique()))
print('POLY ETHV USDC Token Symbol: {}'.format(polygon_ethv_usdc["TokenSymbol"].unique()))

print('ARB BTCV DAI Token Symbol: {}'.format(arbitrum_btcv_dai["TokenSymbol"].unique()))
print('ARB ETHV DAI Token Symbol: {}'.format(arbitrum_ethv_dai["TokenSymbol"].unique()))
print('ARB BTCV USDC Token Symbol: {}'.format(arbitrum_btcv_usdc["TokenSymbol"].unique()))
print('ARB ETHV USDC Token Symbol: {}'.format(arbitrum_ethv_usdc["TokenSymbol"].unique()))

ETH BTCV DAI Token Symbol: ['DAI']
ETH ETHV DAI Token Symbol: ['DAI']
ETH BTCV USDC Token Symbol: ['USDC']
ETH ETHV USDC Token Symbol: ['USDC']
POLY BTCV DAI Token Symbol: ['DAI' 'auto-stake.com' 'NFT Sprites' 'YUI' 'DxDex.io' 'SSX' 'BeezEX'
 'FRUIT']
POLY ETHV DAI Token Symbol: ['DAI' 'auto-stake.com' 'NFT Sprites' 'YUI' 'DxDex.io' 'SSX' 'BeezEX'
 'RicheSwap' 'FRUIT']
POLY BTCV USDC Token Symbol: ['USDC' 'YUI' 'DxDex.io' 'SSX']
POLY ETHV USDC Token Symbol: ['USDC' 'YUI' 'DxDex.io' 'SSX' 'RicheSwap']
ARB BTCV DAI Token Symbol: ['DAI']
ARB ETHV DAI Token Symbol: ['DAI']
ARB BTCV USDC Token Symbol: ['USDC']
ARB ETHV USDC Token Symbol: ['USDC' 'TEST']


### Token Symbol Filtering

Only consider and analyse transactions where the collateral type matches the accepted types of the pool (DAI/USDC)  

There are a number of tokens that show up in the transactions that are not DAI/USDC which at current time of analysis are the only accepted types of collateral in in the volmex protocol pools  

As a result of this, fitler these results out  

Based on a quick-check as shown above some of the token symbols are from projects that no longer exists or are flagged as scams/phishing attempts - either way these transactions are being exlcuded from the analysis

In [130]:
# Ethereum Transaction Symbol Filtering
eth_net_btcv_dai = eth_net_btcv_dai[eth_net_btcv_dai["TokenSymbol"] == 'DAI']
eth_net_ethv_dai = eth_net_ethv_dai[eth_net_ethv_dai["TokenSymbol"] == 'DAI']
eth_net_btcv_usdc = eth_net_btcv_usdc[eth_net_btcv_usdc["TokenSymbol"] == 'USDC']
eth_net_ethv_usdc = eth_net_ethv_usdc[eth_net_ethv_usdc["TokenSymbol"] == 'USDC']

# Polygon Transaction Symbol Filtering
polygon_btcv_dai = polygon_btcv_dai[polygon_btcv_dai["TokenSymbol"] == 'DAI']
polygon_ethv_dai = polygon_ethv_dai[polygon_ethv_dai["TokenSymbol"] == 'DAI']
polygon_btcv_usdc = polygon_btcv_usdc[polygon_btcv_usdc["TokenSymbol"] == 'USDC']
polygon_ethv_usdc = polygon_ethv_usdc[polygon_ethv_usdc["TokenSymbol"] == 'USDC']

# Arbitrum Transaction Symbol Filtering
arbitrum_btcv_dai = arbitrum_btcv_dai[arbitrum_btcv_dai["TokenSymbol"] == 'DAI']
arbitrum_ethv_dai = arbitrum_ethv_dai[arbitrum_ethv_dai["TokenSymbol"] == 'DAI']
arbitrum_btcv_usdc = arbitrum_btcv_usdc[arbitrum_btcv_usdc["TokenSymbol"] == 'USDC']
arbitrum_ethv_usdc = arbitrum_ethv_usdc[arbitrum_ethv_usdc["TokenSymbol"] == 'USDC']

### Transaction Value Formatting

In [131]:
# Replace comma formatting for thousands and cast to type float for Etherscan files
eth_net_btcv_dai["Value"] = eth_net_btcv_dai["Value"].str.replace(',', '').astype(float)
eth_net_ethv_dai["Value"] = eth_net_ethv_dai["Value"].str.replace(',', '').astype(float)
eth_net_btcv_usdc["Value"] = eth_net_btcv_usdc["Value"].str.replace(',', '').astype(float)
eth_net_ethv_usdc["Value"] = eth_net_ethv_usdc["Value"].str.replace(',', '').astype(float)

# Replace comma formatting for thousands and cast to type float for Polygonscan files
polygon_btcv_dai["Value"] = polygon_btcv_dai["Value"].str.replace(',', '').astype(float)
polygon_ethv_dai["Value"] = polygon_ethv_dai["Value"].str.replace(',', '').astype(float)
polygon_btcv_usdc["Value"] = polygon_btcv_usdc["Value"].str.replace(',', '').astype(float)
polygon_ethv_usdc["Value"] = polygon_ethv_usdc["Value"].str.replace(',', '').astype(float)

# Replace comma formatting for thousands and cast to type float for Arbiscan files
arbitrum_btcv_dai["Value"] = arbitrum_btcv_dai["Value"].str.replace(',', '').astype(float)
arbitrum_ethv_dai = arbitrum_ethv_dai["Value"].str.replace(',', '').astype(float)
arbitrum_btcv_usdc = arbitrum_btcv_usdc["Value"].str.replace(',', '').astype(float)
arbitrum_ethv_usdc = arbitrum_ethv_usdc["Value"].str.replace(',', '').astype(float)

### Minters - Combine Data

Combine all of the minters data into a single dataframe to work with for the rest of the notebook

In [132]:
# Combine into single dataframe
minters_df = pd.concat([eth_net_btcv_dai, eth_net_ethv_dai, eth_net_btcv_usdc, eth_net_ethv_usdc, 
                        polygon_btcv_dai, polygon_ethv_dai, polygon_btcv_usdc, polygon_ethv_usdc,
                        arbitrum_btcv_dai, arbitrum_ethv_dai, arbitrum_btcv_usdc, arbitrum_ethv_usdc], ignore_index=True)

In [133]:
# Write to CSV after filtering and number formatting
minters_df.to_csv('minters_v4_final.csv')

In [134]:
# Filter for collateralize events and addresses - volmex works by minting volatility tokens to users that deposit collateral
# Therefore the minters are addresses that have deposited collateral into the protocol
minter_addresses_df = minters_df[(minters_df['type'] == 'COLLATERALIZE')]

## Unique Minters

In [135]:
# Number of unique minters overall and across all 3 chains
print('Number of unique addresses that minted volatility tokens: {}'.format(minter_addresses_df['From'].nunique()))

Number of unique addresses that minted volatility tokens: 5595


## Unique Minters Split by Chain

In [136]:
minter_addresses_by_chain_df = minter_addresses_df.groupby(by='chain', as_index=False).agg({'From': pd.Series.nunique})

In [137]:
print("ETHEREUM: Number of unique addresses that minted volatility tokens: {}".format(minter_addresses_by_chain_df[minter_addresses_by_chain_df["chain"] == 'ETHEREUM']["From"].values[0]))
print("POLYGON: Number of unique addresses that minted volatility tokens: {}".format(minter_addresses_by_chain_df[minter_addresses_by_chain_df["chain"] == 'POLYGON']["From"].values[0]))
print("ARBITRUM: Number of unique addresses that minted volatility tokens: {}".format(minter_addresses_by_chain_df[minter_addresses_by_chain_df["chain"] == 'ARBITRUM']["From"].values[0]))

ETHEREUM: Number of unique addresses that minted volatility tokens: 764
POLYGON: Number of unique addresses that minted volatility tokens: 4724
ARBITRUM: Number of unique addresses that minted volatility tokens: 135


## Minters Bucketing

In [138]:
# Histogram for number of times interacting wtih protocol and values
minter_address_counts_df = pd.Series(minter_addresses_df["From"].value_counts(), name='Cumulative Number of Mints').to_frame()

### Cumulative Mints by User

The histogram below shows the cumulative number of mints (the number of mints that an address has performed) of each user.  

The x-axis "Cumulative Number of Mints" shows the groupings (bins) of the users and the y-axis "count" shows the number of unique users that fall into this grouping.

**Example**: At Cumulative Number of Mints = 1 on the x-axis, the count value is 4695 - this means that there are 4695 users that have performed a Mint once  

The purpose of the histogram is to enable an overall view on the Cumulative Mints performed by each user across the dataset

In [139]:
fig = px.histogram(
    minter_address_counts_df, 
    title='Histogram of Cumulative Mints by User',
    x='Cumulative Number of Mints', 
    marginal="rug")
fig.show()

In [140]:
# Label data with grouping low/medium/high to enable joining back onto CSV for minters
cut_labels_3 = ['Low', 'Medium', 'High']
cut_bins = [0, 2, 7, 100]
minter_address_counts_df['cumulative_mint_groupings'] = pd.cut(minter_address_counts_df['Cumulative Number of Mints'], bins=cut_bins, labels=cut_labels_3)

In [141]:
minter_address_counts_df.head()

,Cumulative Number of Mints,cumulative_mint_groupings
0x74bc67ed6948f0a4c387c353975f142dc640537a,21,High
0x643eed6ce2507e0e7ec6d37d487159661e4e4aa8,10,High
0x66bffe06feba47cd1c95fc2635933c1315c7d2f1,7,Medium
0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,7,Medium
0x3c8d1abf82599f54519e477e194199720b06541f,6,Medium


In [142]:
minter_address_counts_df = minter_address_counts_df.reset_index().rename(columns={"index": "From"})

In [143]:
minter_addresses_grouped_df = pd.merge(minter_addresses_df,minter_address_counts_df,on='From',how='left')
minter_addresses_grouped_df.head()

,0,ContractAddress,DateTime,From,To,TokenName,TokenSymbol,Txhash,UnixTimestamp,Value,chain,type,Cumulative Number of Mints,cumulative_mint_groupings
0,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 18:00:22,0x643eed6ce2507e0e7ec6d37d487159661e4e4aa8,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xfccc42304e6932516389b4799afe7060109ed19e461c...,1.623434e+09,25.0,ETHEREUM,COLLATERALIZE,10,High
1,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 18:09:14,0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xb5cd641ab2f0063f879e0f22bef52fd40b01fe7ee022...,1.623435e+09,1000.0,ETHEREUM,COLLATERALIZE,7,Medium
2,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 19:29:20,0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xc632ede6607ed810d58cdcf0ae86a1f3c23db4db6bc1...,1.623440e+09,597.0,ETHEREUM,COLLATERALIZE,7,Medium
3,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-15 14:51:07,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xe7b024d655837f6598f3f0b5086357322a374dbb5d27...,1.623769e+09,10000.0,ETHEREUM,COLLATERALIZE,2,Low
4,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-16 00:29:50,0x4839c3d305516b6c56647b27b6b70254f0e89cb8,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0x3af79bbf70d8d5712f60b4f895e17288bdca37a6a637...,1.623803e+09,50.0,ETHEREUM,COLLATERALIZE,1,Low


In [144]:
minter_addresses_grouped_df_mints_df = minter_addresses_grouped_df.groupby(by='cumulative_mint_groupings', as_index=False).agg({'From': pd.Series.nunique}).rename(columns={"From": "Number of Users"})
minter_addresses_grouped_df_mints_df.head()

,cumulative_mint_groupings,Number of Users
0,Low,5438
1,Medium,155
2,High,2


### Histogram of Total Value Collateralized by User

The histogram below shows the Total Value Collateralized by User in Volmex.  

The x-axis shows the Total Value Collateralized (in $ - the only tokens allowed for collateral are DAI & USDC so let's assume an exact peg to the dollar even though practically there can be some differences but for this analysis this should be fine)  

The User's are then categorized into Low, Medium and High based on the Total Value they have Collateralized

Low - 0 - \$750  
Medium - \$750-\$1750  
High - \$1750 - Max amount   

In [145]:
collateral_address_counts_df = pd.Series(minter_addresses_grouped_df.groupby(['From'])['Value'].sum(), name='Total Value Collateralized ($)').to_frame()

In [146]:
fig = px.histogram(
    collateral_address_counts_df,
    title='Histogram of Total Value Collateralized by User $',
    x='Total Value Collateralized ($)', 
    marginal="rug")
fig.show()

In [147]:
# Label data with grouping low/medium/high to enable joining back onto CSV for minters
cut_labels_3 = ['Low', 'Medium', 'High']
cut_bins = [0, 750, 1750, 300000]
collateral_address_counts_df['total_value_collateralized_grouping'] = pd.cut(collateral_address_counts_df['Total Value Collateralized ($)'], bins=cut_bins, labels=cut_labels_3)

In [148]:
collateral_address_counts_df = collateral_address_counts_df.reset_index().rename(columns={"index": "From"})
collateral_addresses_grouped_df = pd.merge(minter_addresses_grouped_df, collateral_address_counts_df, on="From", how='left')
collateral_addresses_grouped_df.head()

,0,ContractAddress,DateTime,From,To,TokenName,TokenSymbol,Txhash,UnixTimestamp,Value,chain,type,Cumulative Number of Mints,cumulative_mint_groupings,Total Value Collateralized ($),total_value_collateralized_grouping
0,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 18:00:22,0x643eed6ce2507e0e7ec6d37d487159661e4e4aa8,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xfccc42304e6932516389b4799afe7060109ed19e461c...,1.623434e+09,25.0,ETHEREUM,COLLATERALIZE,10,High,1171.3139,Medium
1,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 18:09:14,0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xb5cd641ab2f0063f879e0f22bef52fd40b01fe7ee022...,1.623435e+09,1000.0,ETHEREUM,COLLATERALIZE,7,Medium,6890.0000,High
2,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 19:29:20,0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xc632ede6607ed810d58cdcf0ae86a1f3c23db4db6bc1...,1.623440e+09,597.0,ETHEREUM,COLLATERALIZE,7,Medium,6890.0000,High
3,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-15 14:51:07,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xe7b024d655837f6598f3f0b5086357322a374dbb5d27...,1.623769e+09,10000.0,ETHEREUM,COLLATERALIZE,2,Low,20000.0000,High
4,NaN,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-16 00:29:50,0x4839c3d305516b6c56647b27b6b70254f0e89cb8,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0x3af79bbf70d8d5712f60b4f895e17288bdca37a6a637...,1.623803e+09,50.0,ETHEREUM,COLLATERALIZE,1,Low,50.0000,Low


In [149]:
collateral_addresses_grouped_df = collateral_addresses_grouped_df.drop([0], axis=1)
collateral_addresses_grouped_df.head()

,ContractAddress,DateTime,From,To,TokenName,TokenSymbol,Txhash,UnixTimestamp,Value,chain,type,Cumulative Number of Mints,cumulative_mint_groupings,Total Value Collateralized ($),total_value_collateralized_grouping
0,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 18:00:22,0x643eed6ce2507e0e7ec6d37d487159661e4e4aa8,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xfccc42304e6932516389b4799afe7060109ed19e461c...,1.623434e+09,25.0,ETHEREUM,COLLATERALIZE,10,High,1171.3139,Medium
1,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 18:09:14,0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xb5cd641ab2f0063f879e0f22bef52fd40b01fe7ee022...,1.623435e+09,1000.0,ETHEREUM,COLLATERALIZE,7,Medium,6890.0000,High
2,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-11 19:29:20,0x27c65f6767b7e020d5d64fc70cf99cf9c56e1286,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xc632ede6607ed810d58cdcf0ae86a1f3c23db4db6bc1...,1.623440e+09,597.0,ETHEREUM,COLLATERALIZE,7,Medium,6890.0000,High
3,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-15 14:51:07,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0xe7b024d655837f6598f3f0b5086357322a374dbb5d27...,1.623769e+09,10000.0,ETHEREUM,COLLATERALIZE,2,Low,20000.0000,High
4,0x6b175474e89094c44da98b954eedeac495271d0f,2021-06-16 00:29:50,0x4839c3d305516b6c56647b27b6b70254f0e89cb8,0x187922d4235d10239b2c6ccb2217ada724f56dda,Dai Stablecoin,DAI,0x3af79bbf70d8d5712f60b4f895e17288bdca37a6a637...,1.623803e+09,50.0,ETHEREUM,COLLATERALIZE,1,Low,50.0000,Low


In [150]:
collateral_addresses_grouped_collateral_df = collateral_addresses_grouped_df.groupby(by='total_value_collateralized_grouping', as_index=False).agg({'From': pd.Series.nunique}).rename(columns={"From": "Number of Users"})
collateral_addresses_grouped_collateral_df.head()

,total_value_collateralized_grouping,Number of Users
0,Low,5092
1,Medium,387
2,High,116


In [151]:
# Write minters groupings to CSV
collateral_addresses_grouped_df.to_csv('minters_v4_final_grouped.csv')

## Liquidity Providers and Traders

In [152]:
# Read ethereum LP/Trader CSV files
eth_net_btcv_usdc = pd.read_csv('/content/btcv_usdc_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
eth_net_ethv_usdc = pd.read_csv('/content/ethv_usdc_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
eth_net_ibtcv_usdc = pd.read_csv('/content/ibtcv_usdc_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
eth_net_iethv_usdc = pd.read_csv('/content/iethv_usdc_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])

# Read polygon LP/Trader CSV files
polygon_btcv_usdc = pd.read_csv('/content/btcv_usdc_quickswap_v3 (1).csv', index_col=False).drop(columns=['Unnamed: 0'])
polygon_ethv_usdc = pd.read_csv('/content/ethv_usdc_quickswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
polygon_ibtcv_usdc = pd.read_csv('/content/ibtcv_usdc_quickswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
polygon_iethv_usdc = pd.read_csv('/content/iethv_usdc_quickswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])

# Read arbitrum LP/Trader CSV files
arbitrum_btcv_usdc = pd.read_csv('/content/btcv_usdc_arbitrum_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
arbitrum_ethv_usdc = pd.read_csv('/content/ethv_usdc_arbitrum_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
arbitrum_ibtcv_usdc = pd.read_csv('/content/ibtcv_usdc_arbitrum_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])
arbitrum_iethv_usdc = pd.read_csv('/content/iethv_usdc_arbitrum_uniswap_v3.csv', index_col=False).drop(columns=['Unnamed: 0'])

In [153]:
# Add Chain Type
eth_net_btcv_usdc['chain'] = 'ETHEREUM'
eth_net_ethv_usdc['chain'] = 'ETHEREUM'
eth_net_ibtcv_usdc['chain'] = 'ETHEREUM'
eth_net_iethv_usdc['chain'] = 'ETHEREUM'

polygon_btcv_usdc['chain'] = 'POLYGON'
polygon_ethv_usdc['chain'] = 'POLYGON'
polygon_ibtcv_usdc['chain'] = 'POLYGON'
polygon_iethv_usdc['chain'] = 'POLYGON'

arbitrum_btcv_usdc['chain'] = 'ARBITRUM'
arbitrum_ethv_usdc['chain'] = 'ARBITRUM'
arbitrum_ibtcv_usdc['chain'] = 'ARBITRUM'
arbitrum_iethv_usdc['chain'] = 'ARBITRUM'

### Traders and Liquidity Providers - Process and Format Data

In [154]:
# Check the TokenSymbols that are part of the ERC20 Transactions
print('ETH BTCV USDC Token Symbol: {}'.format(eth_net_btcv_usdc["TokenSymbol"].unique()))
print('ETH ETHV USDC Token Symbol: {}'.format(eth_net_ethv_usdc["TokenSymbol"].unique()))
print('ETH iBTCV USDC Token Symbol: {}'.format(eth_net_ibtcv_usdc["TokenSymbol"].unique()))
print('ETH iETHV USDC Token Symbol: {}'.format(eth_net_iethv_usdc["TokenSymbol"].unique()))

print('POLY BTCV USDC Token Symbol: {}'.format(polygon_btcv_usdc["TokenSymbol"].unique()))
print('POLY ETHV USDC Token Symbol: {}'.format(polygon_ethv_usdc["TokenSymbol"].unique()))
print('POLY iBTCV USDC Token Symbol: {}'.format(polygon_ibtcv_usdc["TokenSymbol"].unique()))
print('POLY iETHV USDC Token Symbol: {}'.format(polygon_iethv_usdc["TokenSymbol"].unique()))

print('ARB BTCV USDC Token Symbol: {}'.format(arbitrum_btcv_usdc["TokenSymbol"].unique()))
print('ARB ETHV USDC Token Symbol: {}'.format(arbitrum_ethv_usdc["TokenSymbol"].unique()))
print('ARB iBTCV USDC Token Symbol: {}'.format(arbitrum_ibtcv_usdc["TokenSymbol"].unique()))
print('ARB iETHV USDC Token Symbol: {}'.format(arbitrum_iethv_usdc["TokenSymbol"].unique()))

ETH BTCV USDC Token Symbol: ['BTCV' 'USDC']
ETH ETHV USDC Token Symbol: ['USDC' 'ETHV']
ETH iBTCV USDC Token Symbol: ['iBTCV' 'USDC']
ETH iETHV USDC Token Symbol: ['iETHV' 'USDC']
POLY BTCV USDC Token Symbol: ['BTCV' 'USDC' 'auto-stake.com' 'DxDex.io' 'UNI-V2']
POLY ETHV USDC Token Symbol: ['ETHV' 'USDC' 'UNI-V2' 'auto-stake.com' 'DxDex.io' 'RicheSwap' 'SSX']
POLY iBTCV USDC Token Symbol: ['iBTCV' 'USDC' 'auto-stake.com' 'DxDex.io' 'UNI-V2']
POLY iETHV USDC Token Symbol: ['iETHV' 'USDC' 'UNI-V2' 'auto-stake.com' 'YUI' 'DxDex.io' 'BeezEX' 'USBL'
 'SSX']
ARB BTCV USDC Token Symbol: ['BTCV' 'USDC']
ARB ETHV USDC Token Symbol: ['ETHV' 'USDC']
ARB iBTCV USDC Token Symbol: ['iBTCV' 'USDC']
ARB iETHV USDC Token Symbol: ['iETHV' 'USDC']


### Token Symbol Filtering

Similar to the minters data, there are some transactions with token symbols that will be filterd out  

Only keep the transactions that are supported for the Uniswap/Quickswap pools  

There was one token symbol outwith a volatility token or USDC that could have been legitimate which was UNI-V2, however after investigation this was discovered to just be interimediate transactions in either swaps or liquidity actions - as a result they are not required for the analysis

In [155]:
# Ethereum Transaction Symbol Filtering
eth_net_btcv_usdc = eth_net_btcv_usdc[eth_net_btcv_usdc["TokenSymbol"].isin(['BTCV', 'USDC'])]
eth_net_ethv_usdc = eth_net_ethv_usdc[eth_net_ethv_usdc["TokenSymbol"].isin(['ETHV', 'USDC'])]
eth_net_ibtcv_usdc = eth_net_ibtcv_usdc[eth_net_ibtcv_usdc["TokenSymbol"].isin(['iBTCV', 'USDC'])]
eth_net_iethv_usdc = eth_net_iethv_usdc[eth_net_iethv_usdc["TokenSymbol"].isin(['iETHV', 'USDC'])]

# Polygon Transaction Symbol Filtering
polygon_btcv_usdc = polygon_btcv_usdc[polygon_btcv_usdc["TokenSymbol"].isin(['BTCV', 'USDC'])]
polygon_ethv_usdc = polygon_ethv_usdc[polygon_ethv_usdc["TokenSymbol"].isin(['ETHV', 'USDC'])]
polygon_ibtcv_usdc = polygon_ibtcv_usdc[polygon_ibtcv_usdc["TokenSymbol"].isin(['iBTCV', 'USDC'])]
polygon_iethv_usdc = polygon_iethv_usdc[polygon_iethv_usdc["TokenSymbol"].isin(['iETHV', 'USDC'])]

# Arbitrum Transaction Symbol Filtering
arbitrum_btcv_usdc = arbitrum_btcv_usdc[arbitrum_btcv_usdc["TokenSymbol"].isin(['BTCV', 'USDC'])]
arbitrum_ethv_usdc = arbitrum_ethv_usdc[arbitrum_ethv_usdc["TokenSymbol"].isin(['ETHV', 'USDC'])]
arbitrum_ibtcv_usdc = arbitrum_ibtcv_usdc[arbitrum_ibtcv_usdc["TokenSymbol"].isin(['iBTCV', 'USDC'])]
arbitrum_iethv_usdc = arbitrum_iethv_usdc[arbitrum_iethv_usdc["TokenSymbol"].isin(['iETHV', 'USDC'])]

### Transaction Value Formatting

In [156]:
# Replace comma formatting for thousands and cast to type float for Etherscan files
eth_net_btcv_usdc["Value"] = eth_net_btcv_usdc["Value"].str.replace(',', '').astype(float)
eth_net_ethv_usdc["Value"] = eth_net_ethv_usdc["Value"].str.replace(',', '').astype(float)
eth_net_ibtcv_usdc["Value"] = eth_net_ibtcv_usdc["Value"].str.replace(',', '').astype(float)
eth_net_iethv_usdc["Value"] = eth_net_iethv_usdc["Value"].str.replace(',', '').astype(float)

# Replace comma formatting for thousands and cast to type float for Polygonscan files
polygon_btcv_usdc["Value"] = polygon_btcv_usdc["Value"].str.replace(',', '').astype(float)
polygon_ethv_usdc["Value"] = polygon_ethv_usdc["Value"].str.replace(',', '').astype(float)
polygon_ibtcv_usdc["Value"] = polygon_ibtcv_usdc["Value"].str.replace(',', '').astype(float)
polygon_iethv_usdc["Value"] = polygon_iethv_usdc["Value"].str.replace(',', '').astype(float)

# Replace comma formatting for thousands and cast to type float for Arbiscan files
arbitrum_btcv_usdc["Value"] = arbitrum_btcv_usdc["Value"].str.replace(',', '').astype(float)
arbitrum_ethv_usdc["Value"] = arbitrum_ethv_usdc["Value"].str.replace(',', '').astype(float)
arbitrum_ibtcv_usdc["Value"] = arbitrum_ibtcv_usdc["Value"].str.replace(',', '').astype(float)
# Not required for the iethv on atrbitrum as there are no commas to replace in the formatting
# Would be required if any iethv values become comma separated in the future (not required just now as there are no transaction values in the thousands)
# arbitrum_iethv_usdc["Value"] = arbitrum_iethv_usdc["Value"].str.replace(',', '').astype(float)

### Traders and Liquidity Providers - Combine Data

Combine all of the traders and liquidity providers data into a single dataframe to work with for the rest of the notebook

In [157]:
# Combine into single dataframe
lps_traders_df = pd.concat([eth_net_btcv_usdc, eth_net_ethv_usdc, eth_net_ibtcv_usdc, eth_net_iethv_usdc, 
                            polygon_btcv_usdc, polygon_ethv_usdc, polygon_ibtcv_usdc, polygon_iethv_usdc,
                            arbitrum_btcv_usdc, arbitrum_ethv_usdc, arbitrum_ibtcv_usdc, arbitrum_iethv_usdc], ignore_index=True)

In [158]:
# Write to CSV
lps_traders_df.to_csv('lps_traders_v4_final.csv')

## Unique Traders

In [159]:
# Filter for addresses that have performed at least 1 swap - these are known as the traders
# Swap action is where one asset was swapped for another, Liqudity Added/Liqudity Removed Swap combination is where a tx_hash includes a liqudity action and a swap however it is difficult to determine which parts are which from the data
# Only swap makes up the majority of the swap transactions
trader_addresses_df = lps_traders_df[lps_traders_df['tx_type_label'] == 'Swap']

In [160]:
# Number of unique traders overall and across all 3 chains
print('Number of unique trader addresses: {}'.format(trader_addresses_df['From'].nunique()))

Number of unique trader addresses: 1391


## Unique Traders Split by Chain

In [161]:
trader_addresses_by_chain_df = trader_addresses_df.groupby(by='chain', as_index=False).agg({'From': pd.Series.nunique})

In [162]:
print("ETHEREUM: Number of unique trader addresses: {}".format(trader_addresses_by_chain_df[trader_addresses_by_chain_df["chain"] == 'ETHEREUM']["From"].values[0]))
print("POLYGON: Number of unique trader addresses: {}".format(trader_addresses_by_chain_df[trader_addresses_by_chain_df["chain"] == 'POLYGON']["From"].values[0]))
print("ARBITRUM: Number of unique trader addresses: {}".format(trader_addresses_by_chain_df[trader_addresses_by_chain_df["chain"] == 'ARBITRUM']["From"].values[0]))

ETHEREUM: Number of unique trader addresses: 147
POLYGON: Number of unique trader addresses: 1212
ARBITRUM: Number of unique trader addresses: 39


## Unique LPs

In [163]:
# Filter for addresses that have perfomed at least 1 liqudity add action - these are known as the LP's
# There are 2 types to look for: Add Liquidity (Address added liquidity to the pool) and Liquidity Added and Swap (Address added liquidity and performed a swap - so include also) however it is difficult to determine which parts are which (the swap and the liquidity add) 
# from the data so just include liquidity added only (these are the majority anyway)
lps_addresses_df = lps_traders_df[lps_traders_df['tx_type_label'] == 'Add Liquidity']

In [164]:
# Number of unique traders overall and across all 3 chains
print('Number of unique LPs addresses: {}'.format(lps_addresses_df['From'].nunique()))

Number of unique LPs addresses: 1685


## Unique LPs Split by Chain

In [165]:
lps_addresses_by_chain_df = lps_addresses_df.groupby(by='chain', as_index=False).agg({'From': pd.Series.nunique})

In [166]:
print("ETHEREUM: Number of unique LPs addresses: {}".format(lps_addresses_by_chain_df[lps_addresses_by_chain_df["chain"] == 'ETHEREUM']["From"].values[0]))
print("POLYGON: Number of unique LPs addresses: {}".format(lps_addresses_by_chain_df[lps_addresses_by_chain_df["chain"] == 'POLYGON']["From"].values[0]))
print("ARBITRUM: Number of unique LPs addresses: {}".format(lps_addresses_by_chain_df[lps_addresses_by_chain_df["chain"] == 'ARBITRUM']["From"].values[0]))

ETHEREUM: Number of unique LPs addresses: 165
POLYGON: Number of unique LPs addresses: 1464
ARBITRUM: Number of unique LPs addresses: 71


## User Bucketing - Traders

In [167]:
# Each swap has 2 components, look at the OUT component and grab the distinct number of addresses
# Eth requires this as some uniswap transactions have intermediate steps (this logic works for polygon and arbitrum)
trader_addresses_filtered_df = trader_addresses_df[trader_addresses_df["Type"] == 'OUT']

In [168]:
trader_addresses_filtered_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain
10,0xc9244e265b8ab7f6812ea48438bc53d2e136b0293e7e...,1623985412,2021-06-18 03:03:32,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x3839acf1ee7699d1f46b1be840d8ad8317fdf757,1.000000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM
12,0xfeb1b87e396a186df9572400d9aeeb1b5b5f8cc014f4...,1624308530,2021-06-21 20:48:50,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x51e15c3f1ab9cf2d28eb8c232e209aa521b45715,885.900502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,OUT,Swap,ETHEREUM
16,0xb696adc0562e8a233a0d87dd47b9bd2a37a9a43bd5d7...,1624457391,2021-06-23 14:09:51,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x81b15c11fd0c736399485307faecd13bdd58185d,0.221364,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM
20,0xf37a1270aca957a61ecdf86b881c1a0b12bc552a25cf...,1624641328,2021-06-25 17:15:28,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x92114563127c16d3d9d459cd1fda5d47110592ae,1.217470,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM
22,0x2b8bf79b533979cb2247854138d97895f8dcc6c1c0ee...,1624895500,2021-06-28 15:51:40,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x66bffe06feba47cd1c95fc2635933c1315c7d2f1,16.267542,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM


In [169]:
trader_address_counts_df = pd.Series(trader_addresses_df["To"].value_counts(), name='Cumulative Number of Trades').to_frame()

### Histogram of Cumulative Trades by User

This histogram shows the cumulative number of trades that have been made by each user.  

The x-axis shows the cumulative number of trades groupings (bins) and the y-axis shows the number of users who have made that number of trades.  

Low: 0-9 Trades  
Medium: 9-40 Trades  
High: 40+ Trades  

In [170]:
fig = px.histogram(
    trader_address_counts_df, 
    x='Cumulative Number of Trades',
    title = 'Histogram of Cumulative Trades by User',
    marginal="rug")
fig.show()

In [171]:
# Label data with grouping low/medium/high to enable joining back onto CSV for minters
cut_labels_3 = ['Low', 'Medium', 'High']
cut_bins = [0, 9, 40, 2000]
trader_address_counts_df['cumulative_trade_groupings'] = pd.cut(trader_address_counts_df['Cumulative Number of Trades'], bins=cut_bins, labels=cut_labels_3)

In [172]:
trader_address_counts_df = trader_address_counts_df.reset_index().rename(columns={"index": "To"})

In [173]:
trader_addresses_grouped_df = pd.merge(trader_addresses_filtered_df,trader_address_counts_df,on='To',how='left')
trader_addresses_grouped_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain,Cumulative Number of Trades,cumulative_trade_groupings
0,0xc9244e265b8ab7f6812ea48438bc53d2e136b0293e7e...,1623985412,2021-06-18 03:03:32,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x3839acf1ee7699d1f46b1be840d8ad8317fdf757,1.000000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,1,Low
1,0xfeb1b87e396a186df9572400d9aeeb1b5b5f8cc014f4...,1624308530,2021-06-21 20:48:50,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x51e15c3f1ab9cf2d28eb8c232e209aa521b45715,885.900502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,OUT,Swap,ETHEREUM,2,Low
2,0xb696adc0562e8a233a0d87dd47b9bd2a37a9a43bd5d7...,1624457391,2021-06-23 14:09:51,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x81b15c11fd0c736399485307faecd13bdd58185d,0.221364,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,1,Low
3,0xf37a1270aca957a61ecdf86b881c1a0b12bc552a25cf...,1624641328,2021-06-25 17:15:28,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x92114563127c16d3d9d459cd1fda5d47110592ae,1.217470,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,1,Low
4,0x2b8bf79b533979cb2247854138d97895f8dcc6c1c0ee...,1624895500,2021-06-28 15:51:40,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x66bffe06feba47cd1c95fc2635933c1315c7d2f1,16.267542,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,50,High


In [174]:
trader_addresses_grouped_cumulative_trades_df = trader_addresses_grouped_df.groupby(by='cumulative_trade_groupings', as_index=False).agg({'To': pd.Series.nunique}).rename(columns={"To": "Number of Users"})
trader_addresses_grouped_cumulative_trades_df.head()

,cumulative_trade_groupings,Number of Users
0,Low,1412
1,Medium,16
2,High,11


## User Bucketing - Total Value Traded

In [175]:
# Standardise the trade values into USDC - there are some trades that will be USDC and some that will be volatility tokens
# The ideal scenario would be to have an up to date table with price history for the volatility tokens however in this case we will use the latest static values from coingecko
# BTCV - $97.44, IBTCV - $175.72, ETHV - $119.94, IETHV - $125.46
def calculate_usd(row):
    if row['TokenSymbol'] == 'BTCV':
        return row['Value'] * 97.44
    elif row['TokenSymbol'] =='iBTCV':
        return row['Value'] * 175.72
    elif row['TokenSymbol'] =='ETHV':
        return row['Value'] * 119.94
    elif row['TokenSymbol'] =='iETHV':
        return row['Value'] * 125.46
    else:
        return row['Value']

trader_addresses_grouped_df['usd_value'] = trader_addresses_grouped_df.apply(calculate_usd, axis=1)

In [176]:
trader_addresses_amounts_df = pd.Series(trader_addresses_grouped_df.groupby(['To'])['usd_value'].sum(), name='Total Value Traded ($)').to_frame()

### Histogram of Total Value Traded ($)

The below is a histogram of Total Value Traded ($).  

The x-axis shows the groupings (bins) of the histogram for the Total Value Traded per user. The y-axis shows the number of users that fall into that grouping.  

Example - In 500-1500 dollar grouping, there are 91 users which means that 91 users that traded in any of the Exchange pools traded a value between 500-1500 dollars  

In [177]:
fig = px.histogram(
    trader_addresses_amounts_df, 
    x='Total Value Traded ($)', 
    title='Histogram of Total Value Traded ($)',
    marginal="rug")
fig.show()

In [178]:
# Label data with grouping low/medium/high to enable joining back onto CSV for minters
cut_labels_3 = ['Low', 'Medium', 'High']
cut_bins = [0, 2000, 10000, 200000]
trader_addresses_amounts_df['total_value_traded_grouping'] = pd.cut(trader_addresses_amounts_df['Total Value Traded ($)'], bins=cut_bins, labels=cut_labels_3)

In [179]:
trader_addresses_amounts_df = trader_addresses_amounts_df.reset_index().rename(columns={"index": "To"})
trader_addresses_amounts_grouped_df = pd.merge(trader_addresses_grouped_df, trader_addresses_amounts_df, on="To", how='left')
trader_addresses_amounts_grouped_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain,Cumulative Number of Trades,cumulative_trade_groupings,usd_value,Total Value Traded ($),total_value_traded_grouping
0,0xc9244e265b8ab7f6812ea48438bc53d2e136b0293e7e...,1623985412,2021-06-18 03:03:32,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x3839acf1ee7699d1f46b1be840d8ad8317fdf757,1.000000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,1,Low,97.440000,97.440000,Low
1,0xfeb1b87e396a186df9572400d9aeeb1b5b5f8cc014f4...,1624308530,2021-06-21 20:48:50,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x51e15c3f1ab9cf2d28eb8c232e209aa521b45715,885.900502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,OUT,Swap,ETHEREUM,2,Low,885.900502,2474.467363,Medium
2,0xb696adc0562e8a233a0d87dd47b9bd2a37a9a43bd5d7...,1624457391,2021-06-23 14:09:51,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x81b15c11fd0c736399485307faecd13bdd58185d,0.221364,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,1,Low,21.569708,21.569708,Low
3,0xf37a1270aca957a61ecdf86b881c1a0b12bc552a25cf...,1624641328,2021-06-25 17:15:28,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x92114563127c16d3d9d459cd1fda5d47110592ae,1.217470,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,1,Low,118.630247,118.630247,Low
4,0x2b8bf79b533979cb2247854138d97895f8dcc6c1c0ee...,1624895500,2021-06-28 15:51:40,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0x66bffe06feba47cd1c95fc2635933c1315c7d2f1,16.267542,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,OUT,Swap,ETHEREUM,50,High,1585.109277,86942.084646,High


In [180]:
trader_addresses_amounts_grouped_traded_amount_df = trader_addresses_amounts_grouped_df.groupby(by='total_value_traded_grouping', as_index=False).agg({'To': pd.Series.nunique}).rename(columns={"To": "Number of Users"})
trader_addresses_amounts_grouped_traded_amount_df

,total_value_traded_grouping,Number of Users
0,Low,1393
1,Medium,27
2,High,19


In [181]:
# Write minters groupings to CSV
trader_addresses_amounts_grouped_df.to_csv('traders_v4_final_grouped.csv')

### Cumulative Trading Volume

In [182]:
cumulative_trading_usd_volume = trader_addresses_amounts_grouped_df['usd_value'].sum()
print ("Cumulative Trading USD Volume: ${}".format(cumulative_trading_usd_volume))

Cumulative Trading USD Volume: $997678.2132998797


In [183]:
cumulative_trading_usd_value_per_chain = trader_addresses_amounts_grouped_df.groupby(by='chain', as_index=False).agg({"usd_value": pd.Series.sum})

print("ETHEREUM: Cumulative Trading USD Value: ${}".format(cumulative_trading_usd_value_per_chain[cumulative_trading_usd_value_per_chain["chain"] == 'ETHEREUM']["usd_value"].values[0]))
print("POLYGON: Cumulative Trading USD Value: ${}".format(cumulative_trading_usd_value_per_chain[cumulative_trading_usd_value_per_chain["chain"] == 'POLYGON']["usd_value"].values[0]))
print("ARBITRUM: Cumulative Trading USD Value: ${}".format(cumulative_trading_usd_value_per_chain[cumulative_trading_usd_value_per_chain["chain"] == 'ARBITRUM']["usd_value"].values[0]))

ETHEREUM: Cumulative Trading USD Value: $780481.8719295093
POLYGON: Cumulative Trading USD Value: $211855.5641159281
ARBITRUM: Cumulative Trading USD Value: $5340.777254442244


## User Bucketing - Liquidity Providers

In [184]:
lps_addresses_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain
0,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,11.52923,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM
1,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,2500.00000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM
4,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,30.00000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM
5,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,755.29052,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM
6,0x7a656f17631c6d05b54d94974d56031d6e12512ee7f7...,1623864014,2021-06-16 17:20:14,0x3dac271d1b36a434880c527a678b6487ac9c1f8c,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0.99900,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM


In [185]:
lps_addresses_counts_df = pd.Series(lps_addresses_df["From"].value_counts(), name='Cumulative Number of Liquidity Added Actions').to_frame()

### Histogram of Cumulative Number of Liquidity Added Actions Per User

The histogram below shows the Cumulative Number of Liquidity Added Actions Per User.  

The x-axis shows the groupings (bins) of the number of Liquidity Added Actions groups and the y-axis shows the counts.  

e.g For 2 Cumulative Number of Liquidity Added Actions, there 1037 users - that is 1037 users in the dataset performed 2 liquidity add actions  

Low - 0-2  
Medium - 2-7  
High - 7-16  

In [186]:
fig = px.histogram(lps_addresses_counts_df, 
                   x='Cumulative Number of Liquidity Added Actions', 
                   title='Histogram of Cumulative Number of Liquidity Added Actions Per User',
                   marginal="rug")
fig.show()

In [187]:
# Label data with grouping low/medium/high to enable joining back onto CSV for minters
cut_labels_3 = ['Low', 'Medium', 'High']
cut_bins = [0, 2, 7, 16]
lps_addresses_counts_df['cumulative_liquidity_added_groupings'] = pd.cut(lps_addresses_counts_df['Cumulative Number of Liquidity Added Actions'], bins=cut_bins, labels=cut_labels_3)

In [188]:
lps_addresses_counts_df = lps_addresses_counts_df.reset_index().rename(columns={"index": "From"})

In [189]:
lps_addresses_counts_grouped_df = pd.merge(lps_addresses_df,lps_addresses_counts_df,on='From',how='left')
lps_addresses_counts_grouped_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain,Cumulative Number of Liquidity Added Actions,cumulative_liquidity_added_groupings
0,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,11.52923,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,16,High
1,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,2500.00000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM,16,High
2,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,30.00000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,16,High
3,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,755.29052,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM,16,High
4,0x7a656f17631c6d05b54d94974d56031d6e12512ee7f7...,1623864014,2021-06-16 17:20:14,0x3dac271d1b36a434880c527a678b6487ac9c1f8c,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0.99900,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,8,High


In [190]:
lps_addresses_counts_grouped_liquidity_df = lps_addresses_counts_grouped_df.groupby(by='cumulative_liquidity_added_groupings', as_index=False).agg({'From': pd.Series.nunique}).rename(columns={"From": "Number of Users"})
lps_addresses_counts_grouped_liquidity_df.head()

,cumulative_liquidity_added_groupings,Number of Users
0,Low,1040
1,Medium,600
2,High,45


## User Bucketing - Total Liquidity Added Amounts

In [191]:
# Standardise the trade values into USDC - there are some trades that will be USDC and some that will be volatility tokens
# The ideal scenario would be to have an up to date table with price history for the volatility tokens however in this case we will use the latest static values from coingecko
# BTCV - $97.44, IBTCV - $175.72, ETHV - $119.94, IETHV - $125.46
def calculate_usd(row):
    if row['TokenSymbol'] == 'BTCV':
        return row['Value'] * 97.44
    elif row['TokenSymbol'] =='iBTCV':
        return row['Value'] * 175.72
    elif row['TokenSymbol'] =='ETHV':
        return row['Value'] * 119.94
    elif row['TokenSymbol'] =='iETHV':
        return row['Value'] * 125.46
    else:
        return row['Value']

In [192]:
lps_addresses_counts_grouped_df['usd_value'] = lps_addresses_counts_grouped_df.apply(calculate_usd, axis=1)

In [193]:
lps_addresses_counts_grouped_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain,Cumulative Number of Liquidity Added Actions,cumulative_liquidity_added_groupings,usd_value
0,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,11.52923,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,16,High,1123.408199
1,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,2500.00000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM,16,High,2500.000000
2,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,30.00000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,16,High,2923.199998
3,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,755.29052,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM,16,High,755.290520
4,0x7a656f17631c6d05b54d94974d56031d6e12512ee7f7...,1623864014,2021-06-16 17:20:14,0x3dac271d1b36a434880c527a678b6487ac9c1f8c,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0.99900,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,8,High,97.342559


In [194]:
lp_address_amounts_df = pd.Series(lps_addresses_counts_grouped_df.groupby(['From'])['Value'].sum(), name='Cumulative Liquidity Added Value ($)').to_frame()

### Histogram of Cumulative Total Liquidity Added Value

The histogram below shows the Cumulative Total Liquidity Added Value.  

The x-axis shows the groupings (bins) for each Total Liquidity Added Value for the users. The y-axis shows the number of users.  

Example - For the grouping \$500-\$1499, there were 59 users that provided a liquidity value in this range

In [195]:
fig = px.histogram(lp_address_amounts_df, 
                   x='Cumulative Liquidity Added Value ($)', 
                   title='Histogram of Total Liquidity Added Value ($)',
                   marginal="rug")
fig.show()

In [196]:
# Label data with grouping low/medium/high to enable joining back onto CSV for minters
cut_labels_3 = ['Low', 'Medium', 'High']
cut_bins = [0, 1500, 8000, 300000]
lp_address_amounts_df['cumulative_liquidity_added_grouping'] = pd.cut(lp_address_amounts_df['Cumulative Liquidity Added Value ($)'], bins=cut_bins, labels=cut_labels_3)

In [197]:
lp_address_amounts_df = lp_address_amounts_df.reset_index().rename(columns={"index": "From"})
lp_address_amounts_grouped_df = pd.merge(lps_addresses_counts_grouped_df, lp_address_amounts_df, on="From", how='left')
lp_address_amounts_grouped_df.head()

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol,Type,tx_type_label,chain,Cumulative Number of Liquidity Added Actions,cumulative_liquidity_added_groupings,usd_value,Cumulative Liquidity Added Value ($),cumulative_liquidity_added_grouping
0,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,11.52923,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,16,High,1123.408199,26284.587290,High
1,0xc4bc77f254adcbec01372b053782fbf8b58cc84b9084...,1623769680,2021-06-15 15:08:00,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,2500.00000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM,16,High,2500.000000,26284.587290,High
2,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,30.00000,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,16,High,2923.199998,26284.587290,High
3,0x4a63f753cf0578a65bae5127075bb0f5549f66b91507...,1623786369,2021-06-15 19:46:09,0x19ac6292d62465cd3c82a082e0b5f9b875c57d92,0x97693242c5703900fc7651c9164a334ee5ddfa2d,755.29052,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC,IN,Add Liquidity,ETHEREUM,16,High,755.290520,26284.587290,High
4,0x7a656f17631c6d05b54d94974d56031d6e12512ee7f7...,1623864014,2021-06-16 17:20:14,0x3dac271d1b36a434880c527a678b6487ac9c1f8c,0x97693242c5703900fc7651c9164a334ee5ddfa2d,0.99900,0x51b0bcbeff204b39ce792d1e16767fe6f7631970,BTC Volatility Index,BTCV,IN,Add Liquidity,ETHEREUM,8,High,97.342559,461.835293,Low


In [198]:
lp_address_amounts_grouped_liquidity_df = lp_address_amounts_grouped_df.groupby(by='cumulative_liquidity_added_grouping', as_index=False).agg({'From': pd.Series.nunique}).rename(columns={"From": "Number of Users"})
lp_address_amounts_grouped_liquidity_df.head()

,cumulative_liquidity_added_grouping,Number of Users
0,Low,1631
1,Medium,38
2,High,16


In [199]:
# Write minters groupings to CSV
lp_address_amounts_grouped_df.to_csv('lps_v4_final_grouped.csv')